In [ ]:
!pip install requests beautifulsoup4 pandas selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 27.5 MB/s eta 0:00:00


In [ ]:
!pip install webdriver_manager

ERROR: Could not find a version that satisfies the requirement chrome (from versions: none)
ERROR: No matching distribution found for chrome


In [ ]:
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,628 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,845 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,150 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

chrome_options.binary_location = "/usr/bin/chromium-browser"

driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.example.com")
print("Page title:", driver.title)
driver.quit()


Page title: Example Domain


In [ ]:
import time
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL     = 'https://www.shamlola.com'
RECIPES_PATH = '/recipes'
OUTPUT_FILE  = 'recipes.txt'


In [ ]:
def get_listing_soup(page):
    if page == 1:
        resp = requests.get(urljoin(BASE_URL, RECIPES_PATH))
        resp.raise_for_status()
        return BeautifulSoup(resp.text, 'html.parser')
    else:
        params  = {'page': page, 'sortby': 'all', 'type': 'recipe'}
        headers = {'Accept': 'application/json, text/plain, */*'}
        resp = requests.get(urljoin(BASE_URL, RECIPES_PATH), params=params, headers=headers)
        resp.raise_for_status()
        data = resp.json()

        if not isinstance(data, dict) or 'data' not in data:
            raise RuntimeError(f"Unexpected JSON structure on page {page}")

        recipes = data['data'].get('data')
        if not recipes:
            print(f"No more recipes found on page {page}")
            return None

        out = []
        for r in recipes:
            title = r.get('name')
            slug  = r.get('slug')
            if title and slug:
                link = urljoin(BASE_URL, f"/recipes/{slug}")
                out.append({'title': title, 'link': link})
        return out

def scrape_listing(page):
    result = get_listing_soup(page)
    if result is None:
        return []
    if isinstance(result, list):
        return result

    soup = result
    container = soup.find('div', id='recipes-box') or soup
    out = []
    for box in container.find_all('div', class_='custom-grid-ads__box'):
        a = box.select_one('a.news-theme__link[href][title]')
        if not a:
            continue
        out.append({
            'title': a['title'].strip(),
            'link':  urljoin(BASE_URL, a['href'])
        })
    return out

def fetch_recipe_details(url):
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    for tag in soup.find_all('script', type='application/ld+json'):
        try:
            data = json.loads(tag.string)
        except Exception:
            continue
        objs = data if isinstance(data, list) else [data]
        for obj in objs:
            if obj.get('@type') == 'Recipe':
                ingredients = obj.get('recipeIngredient', [])
                instrs = obj.get('recipeInstructions', [])
                steps = []
                for step in instrs:
                    steps.append(step.get('text','').strip() if isinstance(step, dict) else step.strip())
                return {'ingredients': ingredients, 'instructions': steps}

    ingredients = []
    for header in soup.find_all(['h2', 'h3', 'strong']):
        if header.text.strip().startswith("المقادير"):
            ul = header.find_next('ul')
            if not ul:
                ul = header.find_next('div', class_='ingredients')
            if ul:
                for li in ul.find_all('li'):
                    txt = li.get_text(strip=True)
                    if txt:
                        ingredients.append(txt)
            break

    block = soup.find('article') or soup.select_one('.entry-content')
    if block:
        paras = [p.get_text(strip=True) for p in block.find_all('p') if p.get_text(strip=True)]
        paras = [p for p in paras if len(p) > 20 and not p.strip().isdigit()]
        return {'ingredients': ingredients if ingredients else None, 'instructions': paras}

    return {
        'ingredients': ingredients if ingredients else None,
        'instructions': [soup.get_text(separator='\n', strip=True)]
    }


def save_recipe_to_file(recipe):
    with open(OUTPUT_FILE, 'a', encoding='utf-8') as f:
        f.write(f"Title: {recipe['title']}\n")
        f.write("Ingredients: " + (", ".join(recipe['ingredients']) if recipe['ingredients'] else "N/A") + "\n")
        f.write("Instructions:\n")
        for i, step in enumerate(recipe['instructions'], 1):
            f.write(f"{i}. {step}\n")
        f.write("\n" + "-"*50 + "\n\n")

def scrape_all_recipes(delay=0.5):
    page = 1
    all_recipes = []
    while True:
        print(f"→ Scraping listing page #{page}")
        try:
            listings = scrape_listing(page)
        except Exception as e:
            print(f"Error scraping page {page}: {e}")
            break

        if not listings:
            print("No more recipes. Done.")
            break

        for rec in listings:
            print(f"   ⋙ Fetching details for “{rec['title']}”")
            try:
                rec.update(fetch_recipe_details(rec['link']))
                save_recipe_to_file(rec)
                all_recipes.append(rec)
                time.sleep(delay)
            except Exception as e:
                print(f"Failed to fetch {rec['title']}: {e}")
                continue

        page += 1

    return all_recipes


In [ ]:
if __name__ == '__main__':
    open(OUTPUT_FILE, 'w', encoding='utf-8').close()
    recipes = scrape_all_recipes()
    print(f"\nFetched and saved {len(recipes)} recipes.\n")


Streaming output truncated to the last 5000 lines.
   ⋙ Fetching details for “كيك جوز الهند مع الفواكه للكريسماس ”
   ⋙ Fetching details for “للدايت بطاطس محشية لحمة مفرومة أو تونة لباكينام عبدون”
   ⋙ Fetching details for “طريقة عمل كاليماري محشي”
   ⋙ Fetching details for “طريقة عمل صلصة تارتار للسمك”
   ⋙ Fetching details for “كريم برولية لخبيرة الطهى باكينام عبدون”
   ⋙ Fetching details for “طريقة عمل ديك رومي بالصويا والبابريكا”
→ Scraping listing page #1282
   ⋙ Fetching details for “طريقة عمل البــاسـتـا الإيطالية”
   ⋙ Fetching details for “طريقة عمل مناقيش الجبن”
   ⋙ Fetching details for “كيك العسل الأسود للشيف أمنية طه”
   ⋙ Fetching details for “طريقة عمل تشيز كيك للعام الجديد”
   ⋙ Fetching details for “طريقة عمل تارت بسكويت الشيكولاتة للعام الجديد”
   ⋙ Fetching details for “طريقة عمل قطع دجاج بالجبن”
   ⋙ Fetching details for “طريقة عمل فــتـــة الــحــمــص”
   ⋙ Fetching details for “طريقة عمل فطيرة الفراولة بالباف باستري”
   ⋙ Fetching details for “طريقة عمل آيس كريم ب